<a href="https://colab.research.google.com/github/Rahullnitp/Restaurant_recommender/blob/master/Copy_of_final_recommender1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np 
import pandas as pd 
import re
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import gc
import json
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer


databusiness=pd.read_csv('/content/drive/My Drive/recommender/df.csv')

yelp_data=pd.read_csv('/content/drive/My Drive/recommender/yelpData.csv')


import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))
def text_process(mess):                                                                                                                                                                                                                                                                              
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])
userid_df = yelp_data[['user_id','text']]
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=3200)
userid_vectorizer.fit_transform(userid_df['text'])
import pickle

pickle_in = open("/content/drive/My Drive/recommender/P_Q_new.pkl","rb")
P= pickle.load(pickle_in)
Q=pickle.load(pickle_in)
# userid_vectorizer=pickle.load(pickle_in)
pickle_in.close()

def weighted_rating(x,C,m):
  v=x['RatingCount']
  R=x['stars']
 
  w_r=(v/(v+m))*R + (m/(v+m))*C
  # w_r.sort_values(by=['score'], inplace=True)
  return w_r

def User_Rating(user_id,CITY):
  flag=0

  if user_id in P.index:
    test_v_df1=pd.DataFrame(P.loc[P.index==user_id])
    test_v_df1=test_v_df1.reset_index(drop=True)

    predictItemRating=pd.DataFrame(np.dot(test_v_df1.loc[0],Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)
    to_return=[]
    for i in topRecommendations.index:
      if flag==5:
        break
      if databusiness[databusiness['business_id']==i]['city'].any()==CITY:

        flag=flag+1
        to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'RatingCount':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })      
    
    if flag==0:
      print('no restaurant in the city')
    return to_return
  else:
    weighted_data=databusiness.loc[databusiness['city']==CITY]
    C=weighted_data['stars'].mean()
    m=weighted_data['RatingCount'].quantile(0.90)
    qualified_rest=weighted_data.copy().loc[weighted_data['RatingCount'] >= m]  

    qualified_rest['score'] =weighted_rating(qualified_rest,C,m)
    # qualified_rest['score'] = qualified_rest.apply(weighted_rating, axis=1)
    qualified_rest.sort_values(by=['score'], inplace=True,ascending=False)
    # qualified_rest=pd.DataFrame(qualified_rest).drop(['business_id'],axis=1).reset_index(drop=True)
    qualified_rest=pd.DataFrame(qualified_rest).reset_index(drop=True)
    to_return=[]
    for i in qualified_rest.business_id:
      if flag==5:
        break
      if databusiness[databusiness['business_id']==i]['city'].any()==CITY:

        flag=flag+1
        to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'RatingCount':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })      
    

    return to_return

def new_user(CITY,words):
  flag=0
  # words = input("Enter your value: ") 
  test_df= pd.DataFrame([words], columns=['text'])
  test_df['text'] = test_df['text'].apply(text_process)
  test_vectors = userid_vectorizer.transform(test_df['text'])  
  test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())
  to_return=[]
  predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
  topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)

  for i in topRecommendations.index:
    if flag==5:
        break
    if databusiness[databusiness['business_id']==i]['city'].any()==CITY:
      # print(databusiness[databusiness['business_id']==i]['name'].iloc[0])
      # print(databusiness[databusiness['business_id']==i]['categories'].iloc[0])
      # print(str(databusiness[databusiness['business_id']==i]['stars'].iloc[0])+ ' '+str(databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]))
      # print('')
      flag=flag+1
      to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'RatingCount':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })  
  return to_return


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
new_user('Ranchi','Indian food')

[{'Categories': 'Food Stands, Restaurants, Food, Street Vendors',
  'RatingCount': 6,
  'Restaurant': "Goody Woody's",
  'rating': 4.0},
 {'Categories': 'Restaurants, Hot Dogs',
  'RatingCount': 8,
  'Restaurant': "Windy City Beefs 'N Dogs",
  'rating': 3.5},
 {'Categories': 'Lounges, Bars, Nightlife, Tobacco Shops, Sports Bars, Beer, Wine & Spirits, Shopping, Food',
  'RatingCount': 10,
  'Restaurant': 'Desert Rose Cigar Lounge and Club',
  'rating': 3.5},
 {'Categories': 'Seafood, Restaurants, Cajun/Creole, Sandwiches',
  'RatingCount': 74,
  'Restaurant': 'Battiste & Dupree Cajun Grill',
  'rating': 4.0},
 {'Categories': 'Chinese, Restaurants',
  'RatingCount': 200,
  'Restaurant': 'Yao Fine Chinese Cuisine',
  'rating': 4.0}]

In [0]:
User_Rating('QuZbJquRtbY9O9JrB9NpxQ','Ranchi')

[{'Categories': 'Food, Coffee & Tea',
  'RatingCount': 93,
  'Restaurant': 'The Coffee Bean & Tea Leaf',
  'rating': 4.0},
 {'Categories': 'Cheesesteaks, Sandwiches, Restaurants, American (Traditional), Delis',
  'RatingCount': 444,
  'Restaurant': "Philly's Famous",
  'rating': 4.5},
 {'Categories': 'Food, Juice Bars & Smoothies, Coffee & Tea',
  'RatingCount': 82,
  'Restaurant': 'Dutch Bros Coffee',
  'rating': 4.0},
 {'Categories': 'Nightlife, Wine Bars, Restaurants, Canadian (New), Bars',
  'RatingCount': 108,
  'Restaurant': 'Le Vin Papillon',
  'rating': 4.5},
 {'Categories': 'Restaurants, Chinese, Asian Fusion',
  'RatingCount': 416,
  'Restaurant': "Tott's Asian Diner",
  'rating': 4.0}]